# Spark 高级文本处理技术
## 处理文本数据有什么特别之处
文本数据处理的复杂性源于两个原因。第一，文本和语言有隐含的结构信息，使用原始的文本很难捕捉到（例如，含义、上下文、不同词性到词语，句法结构和不同的语言，这些都是明显的几个方面。）因此，单纯的特征提取方法常常没有太大效果。  
第二，文本数据的有效维度一般都非常巨大甚至是无限的。试想一下英语中的单词、所有特殊词、字符、俗语等地总数有多少，然后加上其他语言和所有可以中互联网上找到的文本。因此，即使在较小的数据集上，文本数据按照单词得到的维度也可以轻易超过十万甚至数百万。
## 从数据中抽取合适的特征
自然语言处理（NLP）领域研究文本处理的技术包含提取特征、建模和机器学习。  
我们重点研究MLLib包含的两种特征提取技术：TF－IDF 短语加权表示和特征哈希。
### 短语加权表示
TF－IDF（词频－逆文本词频）给一段文本中每一个词赋予一个权值。这个权值是基于单词在文本中出现的频率计算得到的。同时还要应用逆文本频率做全局归一化。逆文本频率是基于单词在所有文档中的频率计算得到的。（一个单词在文档中出现的次数越多说明权重越重要，在越多的文档中出现的越频繁那么就越没有代表性）
### 特征哈希
特征哈希是一种处理高维数据的技术，并经常被应用中文本和分类数据集上，这些数据集的特征可以取很多不同的值。我们经常使用k分之一编码方法处理包括文本的分类特征。这种方法有效简单，但是对于非常高维的数据却不易使用。
### 从20新闻组数据集中提取TF－IDF特征  
我们使用一个非常有名的数据集20 Newsgroups；这个数据集一般用来做文本分类。这是一个由20个不同主题的新闻组消息组成的集合，有很多种不同的数据格式。对于我们的任务来说，可以使用按日期组织的数据集。数据集 http://qwone.com/~jason/20Newsgroups。  
我们把数据集分为60%训练集和40%的测试集。
#### 1、分析20Newsgroups数据
查看目录结构，我们的数据以独立文件的形式存在（每个文件一条信息）。因此，我们使用wholeTextFiles方法来吧每个文件的内容读取到RDD的每一个纪录中。可以看到文件总数：

In [1]:
val PATH = "file:///Users/lzz/work/SparkML/"

In [2]:
val rdd = sc.wholeTextFiles( PATH + "data/20news-bydate-train/*" )
val text = rdd.map{ case( file, text ) => text }
println( text.count )

11314

然后我们看一下得到的新闻组主题，输出如下：

In [4]:
val newsgroups = rdd.map{ case (file, text) => file.split("/").takeRight(2).head }
val countByGroup = newsgroups.map( n => (n, 1)).reduceByKey(_+_).collect.sortBy(-_._2).mkString("\n")
println( countByGroup )

(rec.sport.hockey,600)
(soc.religion.christian,599)
(rec.motorcycles,598)
(rec.sport.baseball,597)
(sci.crypt,595)
(rec.autos,594)
(sci.med,594)
(comp.windows.x,593)
(sci.space,593)
(sci.electronics,591)
(comp.os.ms-windows.misc,591)
(comp.sys.ibm.pc.hardware,590)
(misc.forsale,585)
(comp.graphics,584)
(comp.sys.mac.hardware,578)
(talk.politics.mideast,564)
(talk.politics.guns,546)
(alt.atheism,480)
(talk.politics.misc,465)
(talk.religion.misc,377)


#### 2、应用基本的分词方法
我们文本处理流程的第一步就是切分每一个文档的原始内容为多个单词（也叫词项），组成集合。这个过程叫做分词。我们实现最简单的空格分词，并把每个文档的所有单词变为小写，运行结果得到分词之后不同单词的数量：

In [5]:
val text = rdd.map { case (file, text) => text }
val whiteSpaceSplit = text.flatMap( t => t.split(" ").map(_.toLowerCase) )
println( whiteSpaceSplit.distinct.count )

402978


即使使用相对较小的文本集，不同单词的个数（也就是我们特征向量的维度）也可能会非常高。  
我们随机选择的查看一篇文档：

In [6]:
println( whiteSpaceSplit.sample(true, 0.3, 42).take(100).mkString(",") )

from:,mathew,mathew,faq:,faq:,atheist,resources
summary:,music,--,fiction,,mantis,consultants,,uk.
supersedes:,290

archive-name:,1.0

,,,,,,,,,,,,,,,,,,,organizations

,organizations

,,,,,,,,,,,,,,,,stickers,and,and,the,from,from,in,to:,to:,ffrf,,256-8900

evolution,designs

evolution,a,stick,cars,,written
inside.,fish,us.

write,evolution,,,,,,,bay,can,get,get,,to,the
price,is,of,the,the,so,on.,and,foote.,,atheist,pp.,0-910309-26-4,,,atrocities,,foote:,aap.,,the


#### 3、改进分词效果
之前简单的分词方法产生了很多单词，而且许多不是单词的字符（比如标点符号）没有过滤掉。大部分分词方案都会把这些字符移除。我们可以使用正则表达式切分原始文档来移除这些非单词字符，这将极大减少不同单词的数量：

In [7]:
val nonWordSplit = text.flatMap( t => t.split( """\W+""" ).map( _.toLowerCase))
println( nonWordSplit.distinct.count )

130126


观察一下前几个单词，我们已经去除来文本中大部分没用的字符：

In [8]:
println( nonWordSplit.distinct.sample( true, 0.3, 42 ).take(100).mkString(",") )

kv07,jejones,jejones,ml5,ml5,w1w3s1,bone,k29p,schwabam,chipset,tenex,hcq,entitlements,he3,neurologists,jxicaijp,regina,regina,b0vp,c1381,adultery,fzbv1u,c1,c1,ao,wargame,nonmeasurable,391k,seetex,viewed,unforeseen,eur,m2ffjk,strut,strut,tic,wp3d,randall_clark,mswin,cannibal,searflame,34ij,13963,13963,siumv,right,z0ozk,z0ozk,g85,93864,igpp,ulcerative,ulcerative,remailing,012537,012537,f80,xs9,xs9,detergent,aanp,rlg1,robert,deterministic,rockefeller,hour,scramblers,shutdown,1r1d62,mtearle,exhausting,discernible,siiafeid8,qnh1,qnh1,paradijs,abstract,crudely,crudely,tripe,chama,triangulate,iaik,dvbtpuc,herod,herod,nutty,homerific,phoniest,uflkll_00vpcekw15e,transylvania,pdp11,liberated,canonical,c2xjfa,birds,xtappcontext,lamers,8v0,b4r


我们使用非单词正则模式来切分文本的效果不错，但仍然有很多包含数字的单词剩下。在有些情况下，数字会成为文档中的重要内容。但对于我们来说，下一步就是要过滤掉数字和包含数字的单词。  
使用正则模式可以过滤掉和这个模式不匹配的单词，这再次减小了单词集的大小：

In [9]:
val regex = """[^0-9]*""".r
val filterNumbers = nonWordSplit.filter(token => regex.pattern.matcher(token).matches )
println( filterNumbers.distinct.count )

84912


让我们再随机来看另一个过滤完单词后的例子：

In [10]:
println( filterNumbers.distinct.sample( true, 0.3, 42 ).take(100).mkString(",") )

rexlex,jejones,jejones,silikian,silikian,reunion,wuair,schwabam,dpsi,bruns,breath,gottschalk,semites,fowl,afterward,ignore,upo,upo,mowtu,arius,jbis,upsets,aces,aces,majorly,akl,underscored,steaminess,sively,jayson,qsins,historians,initiation,krantz,krantz,jmckinney,nonmeasurable,isv,bellevue,seetex,kjvar,rolled,wakaluk,wakaluk,foghorns,lapsing,tinuing,tinuing,croissant,readjoy,comparing,bippy,bippy,sophia,jaze,jaze,mswin,theoreticians,theoreticians,enlarge,decriminalize,right,vied,internship,keysym,igpp,handful,dtn,nixdorf,miserable,brow,icbz,colina,poking,poking,inre,sjoberg,computational,computational,df,emstation,inviting,jkis_ltd,santiago,mishandles,mishandles,anachronistic,springer,hfd,sublingual,vow,nowadays,multiway,formac,altenhofen,responsbible,fuenfzig,trial,rluap,crudely


#### 4、移除停用词 
停用词是指出现中一个文本集所有文档中很多次的常用词。标准的英语停用词包括and the or but of等。提取文本特征等标准做法是从抽取等词中排除停用词。  
当使用TF－IDF加权时，加权模式已经做了这点。一个停用词总是很低的IDF分数，会有一个很低的TF-IDF权值，因此成为了一个不重要的词。有些时候，对于信息检索和搜索任务，停用词又需要被包含。但是，最好还是在提取特征时移除停用词，因为这可以降低最后特征向量的维度和训练数据的大小。  
查看所有文档中高频的词语，看看有没有需要清除掉停用词：

In [12]:
val tokenCounts = filterNumbers.map( t => (t, 1)).reduceByKey(_+_)
val oreringDesc = Ordering.by[(String, Int), Int](_._2)
println( tokenCounts.top(20)(oreringDesc).mkString("\n"))

(the,146532)
(to,75064)
(of,69034)
(a,64195)
(ax,62406)
(and,57957)
(i,53036)
(in,49402)
(is,43480)
(that,39264)
(it,33638)
(for,28600)
(you,26682)
(from,22670)
(s,22337)
(edu,21321)
(on,20493)
(this,20121)
(be,19285)
(t,18728)


把这些词中的某些词和其他常用词集合成一个停用词集，过滤掉这些词之后就可以看到剩下的单词：

In [13]:
val stopwords = Set( "the","a","an","of","or","in","for","by","on","but","is","not",
"with","as","was","if","they","are","this","and","it","have","from","at","my","be","that","to")
val tokenCountsFilteredStopwords = tokenCounts.filter{ case
(k,v) => !stopwords.contains(k) }
println( tokenCountsFilteredStopwords.top(20)(oreringDesc).mkString("\n"))

(ax,62406)
(i,53036)
(you,26682)
(s,22337)
(edu,21321)
(t,18728)
(m,12756)
(subject,12264)
(com,12133)
(lines,11835)
(can,11355)
(organization,11233)
(re,10534)
(what,9861)
(there,9689)
(x,9332)
(all,9310)
(will,9279)
(we,9227)
(one,9008)


我们将删除那些仅仅含有一个字符的单词。这和我们移除停用词的原因类似。这些单独字符组成的单词不太可能包含太多信息。因此可以删除他们来降低特征维度和模型大小：

In [14]:
val tokenCountsFilteredSize = tokenCountsFilteredStopwords.filter{ case(k, v) => k.size >= 2 }
println( tokenCountsFilteredSize.top(20)(oreringDesc).mkString("\n"))

(ax,62406)
(you,26682)
(edu,21321)
(subject,12264)
(com,12133)
(lines,11835)
(can,11355)
(organization,11233)
(re,10534)
(what,9861)
(there,9689)
(all,9310)
(will,9279)
(we,9227)
(one,9008)
(would,8905)
(do,8674)
(he,8441)
(about,8336)
(writes,7844)


#### 5、基于频率去除单词
在分词的时候，还有一种比较常用的去除单词的方法是去掉在整个文本库中出现频率很低的单词。例如，检查文本库中出现频率最低的单词（注意这里我们使用不同排序方式，返回上升排序的结果）：

In [15]:
val oreringAsc = Ordering.by[(String, Int), Int](-_._2)
println(tokenCountsFilteredSize.top(20)(oreringAsc).mkString("\n"))

(altina,1)
(bluffing,1)
(preload,1)
(lennips,1)
(actu,1)
(vno,1)
(wbp,1)
(donnalyn,1)
(ydag,1)
(mirosoft,1)
(jjjjrw,1)
(harger,1)
(conts,1)
(bankruptcies,1)
(uncompression,1)
(d_nibby,1)
(bunuel,1)
(odf,1)
(swith,1)
(pacified,1)


很多短语在整个文本集中子出现一次，这对于机器学习模型是没有价值，因为这些单词我们没有足够的训练数据。应用另一个过滤函数来排除这些很少出现的单词：

In [16]:
val rareTokens = tokenCounts.filter{ case (k, v) => v < 2 }.map{ case (k, v) => k }.collect.toSet
val tokenCountsFilteredAll = tokenCountsFilteredSize.filter{ case(k, v) => !rareTokens.contains(k) }
println( tokenCountsFilteredAll.top(20)(oreringAsc).mkString("\n"))

(sina,2)
(akachhy,2)
(mvd,2)
(sarkis,2)
(wendel_clark,2)
(relieves,2)
(purposeful,2)
(hizbolah,2)
(wout,2)
(uneven,2)
(senna,2)
(subdivided,2)
(bushy,2)
(feagans,2)
(coretest,2)
(oww,2)
(historicity,2)
(mmg,2)
(margitan,2)
(defiance,2)


查看不同的单词有多少个：

In [17]:
println( tokenCountsFilteredAll.count )

51801


通过这分词流程中应用所有这些过滤步骤，把特征的维度从402978降到了51801。  
现在把过滤逻辑组合到一个函数中，并应用到RDD中的每个文档：

In [18]:
def tokenize( line: String ): Seq[String] = {
    line.split("""\W+""")
    .map( _.toLowerCase )
    .filter( token => regex.pattern.matcher(token).matches )
    .filterNot( token => stopwords.contains(token) )
    .filterNot( token => rareTokens.contains(token) )
    .filter( token => token.size >= 2 )
    .toSeq
}
println( text.flatMap(doc => tokenize(doc)).distinct.count )

51801


把RDD中的每个文档按照下面的方式分词：输出如下（第一篇文档第一部分的分词结果）

In [19]:
val tokens = text.map( doc => tokenize(doc))
println( tokens.first.take(20) )

WrappedArray(mathew, mathew, mantis, co, uk, subject, alt, atheism, faq, atheist, resources, summary, books, addresses, music, anything, related, atheism, keywords, faq)


#### 6、关于提取词干
提取词干在文本处理和分词中比较常用。这是一种把整个单词转换为一个基的形式（叫词根）的方法。  
例如，dogs->dog,walking walker->walk

#### 7、训练TF-IDF模型
现在我们使用MLlib把每篇处理成词项形式的文档以向量形式表达。第一步是使用HashingTF实现，它使用特征哈希把每个输入文本的词项映射为一个词频向量的下标。之后，使用一个全局的IDF向量把词频向量转换为TF－IDF向量。  
每个词项的下标是这个词的哈希值（依次映射到特征向量到某个维度）。词项到值是本身的TF－IDF权重（既词项的频率乘以逆文本频率）。  
首先，引入我们需要的类，创建一个HashingTF实例，传入维度参数dim。默认特征维度是20的20次方，因此我们选择18次方，因为使用50000个单词应该不会产生很多的哈希冲突，而较少的维度占用内存更少并且展示起来更方便：

In [21]:
import org.apache.spark.mllib.linalg.{ SparseVector => SV }
import org.apache.spark.mllib.feature.HashingTF
import org.apache.spark.mllib.feature.IDF
val dim = math.pow(2, 18).toInt
val hashingTF = new HashingTF(dim)
val tf = hashingTF.transform( tokens )
tf.cache

MapPartitionsRDD[50] at map at HashingTF.scala:78

HashingTF的transform函数把每个输入文档（既词项的序列）映射到一个MLlib的Vector对象。我们将调用cache把数据保持在内存来加速之后的操作。 
观察一下转换后的数据的第一个元素：

In [23]:
val v = tf.first.asInstanceOf[SV]
println(v.size)
println( v.values.size)
println( v.values.take(10).toSeq )
println( v.indices.take(10).toSeq )

262144
706
WrappedArray(1.0, 1.0, 1.0, 1.0, 2.0, 1.0, 1.0, 2.0, 1.0, 1.0)
WrappedArray(313, 713, 871, 1202, 1203, 1209, 1795, 1862, 3115, 3166)


我们可以看到每一个词频的稀疏向量的维度是262144.然而向量中的非0项仅仅只有706个。输出的最后两行展示了向量中前几列的下标和词频值。  
现在通过创建新的IDF实例并调用RDD中的fit方法，利用词频向量作文输入来对文库中的每个单词计算逆文本频率。之后使用IDF的transform方法转换频率向量为TF－IDF向量：

In [24]:
val idf = new IDF().fit(tf)
val tfidf = idf.transform(tf)
val v2 = tfidf.first.asInstanceOf[SV]
println( v2.values.size )
println( v2.values.take(10).toSeq )
println( v2.indices.take(10).toSeq )

706
WrappedArray(2.3869085659322193, 4.670445463955571, 6.561295835827856, 4.597686109673142, 8.932700215224111, 5.750365619611528, 2.1871123786150006, 5.520408782213984, 3.4312512246662714, 1.7430324343790569)
WrappedArray(313, 713, 871, 1202, 1203, 1209, 1795, 1862, 3115, 3166)


可以看到非零项的数量改变了（现在是706），词向量的下标也变了。之前向量表示每个单词在文档中出现的频率，而在新的向量表示IDF的加权频率。  
#### 8、分析TF－IDF权重  
接下来，我们观察几个单词的TF－IDF权值，分析一个单词的常用或者极少使用的情况会对TF－IDF值产生什么样的影响。  
首先计算整个文档的TF－IDF最小和最大权值，最小的TF－IDF值是0，最大的是一个非常大的数：

In [25]:
val minMaxVals = tfidf.map{ v =>
    val sv = v.asInstanceOf[SV]
    (sv.values.min, sv.values.max)
}
val globalMinMax = minMaxVals.reduce{ case ( (min1, max1),(min2, max2)) => (math.min(min1, min2), math.max(max1, max2))}
println( globalMinMax )

(0.0,66155.39470409753)


对于之前计算得到的频率最高的几个词的TF－IDF表示进行计算，可以看到TF－IDF加权会对常用词赋予较低的权值。  
如果形成了这个文档的TF-IDF向量表示，会看到下面赋予每个单词的值。注意我们使用了特征哈希，所以将不能再确定这些值分别表达的是哪个向量。但是，这些值说明赋予给这些词的权重相对较低：

In [30]:
val common = sc.parallelize( Seq( Seq("you", "do","we")) )
val tfCommon = hashingTF.transform(common)
val tfidfCommon = idf.transform( tfCommon )
val commonVector = tfidfCommon.first.asInstanceOf[SV]
println(commonVector.values.toSeq)

WrappedArray(0.9965359935704624, 1.3348773448236835, 0.5457486182039175)


现在，让我们对几个不常见出现的单词应用相同的转换。直觉上我们认为这些词和某些话题更相关。  
从下面的结果可以看出，这些词的TF－IDF值确实远远高于那些常用词：

In [31]:
val uncommon = sc.parallelize( Seq( Seq("telescope","legislation","investment") ) )
val tfUncommon = hashingTF.transform( uncommon )
val tfidfUncommon = idf.transform( tfUncommon )
val uncommonVector = tfidfUncommon.first.asInstanceOf[SV]
println( uncommonVector.values.toSeq )

WrappedArray(5.3265513728351666, 5.308532867332488, 5.483736956357579)


##  使用TF-IDF模型
### 20Newsgroups数据集的文本相似度和TF－IDF特征 
可以认为两个文档共有的单词越多相似度越高，反之相似度越低。因为我们通过计算两个向量的点积来计算余弦相似度，而每一个向量都由文档中的单词构成，所以共有单词更多的文档余弦度也会更高  
现在来看TF－IDF如何发挥作用。我们有理由期待使非常不同的文档也可能包含很多相同的常用词（例如停用词）。然而，因为较低的TF－IDF权值，这些单词不会对点积的结果产生较大影响，因此不会对相似度的计算产生太大影响。  
例如，我们预估两个从曲棍球新闻组随机选择比较相似。然后看一下是不是这样：

In [32]:
val hockeyText = rdd.filter{ case( file, text) => file.contains("hockey") }
val hockeyTF = hockeyText.mapValues( doc => hashingTF.transform(tokenize(doc)))
val hockeyTfIdf = idf.transform( hockeyTF.map(_._2))

上面的代码首先过滤原始的输入RDD，使其只包含来自曲棍球话题组的消息。然后使用的分词和词频转换函数。注意使用的transform方法是处理单个文档（形式为Seq[String]）的版本，而不是处理包含所有文档的RDD的版本。  
最后，我们使用IDF转换（使用之前已经基于所有文库计算出来相同的IDF值）。  
有了曲棍球文档向量后，就可以随机选择其中两个向量，并计算它们的余弦相似度（我们会使用Breeze的线性函数，首先把MLLib向量转换成Breeze稀疏向量）：

In [33]:
import breeze.linalg._
val hockey1 = hockeyTfIdf.sample( true, 0.1, 42).first.asInstanceOf[SV]
val breeze1 = new SparseVector( hockey1.indices, hockey1.values, hockey1.size )
val hockey2 = hockeyTfIdf.sample( true, 0.1, 43).first.asInstanceOf[SV]
val breeze2 = new SparseVector( hockey2.indices, hockey2.values, hockey2.size )
val cosineSim = breeze1.dot( breeze2 ) / ( norm( breeze1 ) * norm( breeze2 ) )
println( cosineSim )

0.060250114361164626


这个值看起来太低了，但文本数据中大量唯一的单词总会使特征的有效维度很高。因此，我们可以认为即使两个谈论相同话题的文档也可能有着较少的相同单词，因而会有较低的相似度分数。  
作为对照，我们可以和另一个计算结果做比较，其中一个文档来自曲棍球文档，而另一个文档随机选择自comp.graphics新闻组，使用完全相同的方法：

In [35]:
val graphicsText = rdd.filter { case( file, text) => file.contains("comp.graphics")}
val graphicsTF = graphicsText.mapValues( doc => hashingTF.transform(tokenize(doc)))
val graphicsTfIdf = idf.transform( graphicsTF.map( _._2))
val graphics = graphicsTfIdf.sample( true, 0.1, 42 ).first.asInstanceOf[SV]
val breezeGraphics = new SparseVector( graphics.indices, graphics.values, graphics.size )
val cosineSim2 = breeze1.dot( breezeGraphics) / ( norm(breeze1) * norm(breezeGraphics))
println( cosineSim2 )

0.004664850323792852


最后，相比一篇计算机话题组的文档，一篇运动相关话题组的文档很可能会和曲棍球文档有较高的相似度。但我们希望谈论棒球的文档不应该和谈论曲棍球的文档那么相似。下面通过计算从棒球新闻组随机得到的消息和曲棍球文档的相似度来看看是否如此

In [1]:
val baseballText = rdd.filter{ case(file, text) => file.contains("baseball") }
val baseballTF = baseballText.mapValues( doc => hashingTF.transform(tokenize(doc)))
val baseballTfIdf = idf.transform( baseballTF.map(_._2))
val baseball = baseballTfIdf.sample( true, 0.1, 42).first.asInstanceOf[SV]
val breezeBaseball = new SparseVector(baseball.indices, baseball.values, baseball.size)
val cosineSim3 = breeze1.dot( breezeBaseball ) / (norm(breeze1) * norm(breezeBaseball))
println( cosineSim3 )

Name: Compile Error
Message: <console>:12: error: not found: value rdd
       val baseballText = rdd.filter{ case(file, text) => file.contains("baseball") }
                          ^
StackTrace: 

棒球和曲棍球文档的余弦相似度是0.05.与comop.graphics文档相比已经很高，但是和另一篇曲棍球文档相比则较低：

##  基于20 newsgroups 数据集使用TF-IDF训练文本分类器
在20 Newsgroups的例子中，每一个新闻组的主题就是一个类，我们能使用TF－IDF转换后的向量作文输入训练一个分类器。  
因为我们将要处理的是一个多分类的问题，我们使用MLlib中朴素贝叶斯方法，这种方法支持多分类。第一步，引入要使用的spark类：  
之后，抽取20个主题并把他们转换到类的映射。可以像在k选1编码中那样，给每个类赋于一个数字下标：

In [38]:
import org.apache.spark.mllib.regression.LabeledPoint
import org.apache.spark.mllib.classification.NaiveBayes
import org.apache.spark.mllib.evaluation.MulticlassMetrics

val newsgroupsMap = newsgroups.distinct.collect().zipWithIndex.toMap
val zipped = newsgroups.zip(tfidf)
val train = zipped.map{ case(topic, vector) => LabeledPoint(newsgroupsMap(topic), vector)}
train.cache

MapPartitionsRDD[94] at map at <console>:61

在上面的代码中，从新闻组RDD开始，其中每个元素是一个话题，使用zip函数把它和由TF－IDF向量组成的tfidf RDD 组合。然后对新生成的zipped RDD 中的每个键值对通过映射函数创建一个LabeledPont对象，其中每个label是一个类下标，特征就是TF－IDF向量。  
现在我们有了格式正确的输入RDD，可以简单地把它传到朴素贝叶斯的train 方法中：

In [39]:
val model = NaiveBayes.train( train, lambda = 0.1)

让我们在测试数据集上评估一下模型的性能。我们将从20news－bydate-test文件夹中加载原始的测试数据，然后使用wholeTextFiles 把每一条信息读取为RDD中的纪录。使用和得到newsgroups RDD相同的方法从文件路径中提取类标签：

In [40]:
val testRDD = sc.wholeTextFiles( PATH + "data/20news-bydate-test/*" )
val testLabels = testRDD.map{ case(file, text) => 
    val topic = file.split("/").takeRight(2).head
    newsgroupsMap( topic )
}

  使用和训练数据集相同的方法处理测试数据集中的文本（使用tokenize方法），然后使用词频转换，之后再次使用完全相同的从训练数据中计算得到的IDF，把TF向量转换为TF－IDF向量。最后，合并测试类标签和TF－IDF向量，创建我们的测试RDD［LabeledPoint］：

In [43]:
val testTf = testRDD.map{ case(file, text) =>
    hashingTF.transform( tokenize(text))
}
val testTfIdf = idf.transform( testTf )
val zippedTest = testLabels.zip( testTfIdf )
val test = zippedTest.map{ case( topic, vector) => LabeledPoint(topic, vector) }

我们准备计算预测结果和我们模型的真实类标签。我们将使用RDD为模型来计算准确度和多分类加权F－指标：  
可以看到多分类朴素贝叶斯模型的准确率和召回率均接近80%：

In [44]:
val predictionAndLabel = test.map( p => (model.predict(p.features), p.label))
val accuracy = 1.0 * predictionAndLabel.filter( x => x._1 == x._2 ).count() / test.count()
val metrics = new MulticlassMetrics( predictionAndLabel )
println( accuracy )
println( metrics.weightedFMeasure )

0.7915560276155071
0.7810675969031116


##  评估文本处理技术等作用
### 在20Newsgroups数据集上比较原始特征和处理过的TF－IDF特征
在这个例子中，我们在用空格分词处理后的原始文本上应用哈希单词频率转换。我们将在这些文本上训练模型，并模仿我们对使用TF－IDF特征训练的模型所做的，评估在测试集上的表现：

In [ ]:
val rawTokens = rdd.map{ case(file, text) => text.split(" ") }
val rawTF = rawTokens.map( doc => hashingTF.transform(doc))
val rawTrain = newsgroups.zip(rawTF).map{ case( topic, vector ) => LabeledPoint( newsgroupsMap(topic), vector)}
val rawModel = NaiveBayes.train(rawTrain, lambda = 0.1 )
val rawTestTF = testRDD.map{ case(file, text) => hashingTF.transform(text.split(" ")) }
val rawZippedTest = testLabels.zip( rawTestTF )
val rawTest = rawZippedTest.map{ case (topic, vector) => LabeledPoint(topic, vector)}
val rawPredictionAndLabel = rawTest.map( p => (rawModel.predict(p.features), p.label))
val rawAccuracy = 1.0 * rawPredictionAndLabel.filter( x => x._1 == x._2 ).count() / rawTest.count()
println( rawAccuracy)
val rawMetrics = new MulticlassMetrics(rawPredictionAndLabel)
println(rawMetrics.weightedFMeasure)

结果可能会令人惊讶，尽管准确率和F－指标比那些TF－IDF模型低几个百分点，原始的模型表现其实也很不错。这也部分反映了一个事实，既朴素贝叶斯模型能很好的适用于原始词频格式的数据。

## Word2Vec 模型
### 基于20Newsgroups数据集训练Word2Vec
在Spark中训练一个Word2Vec模型相对简单。我们需要传递一个RDD，其中每一个元素都是一个单词的序列。可以使用我们之前得到的分词后的文档来作为模型的输入：

In [ ]:
import org.apache.spark.mllib.feature.Word2Vec
val word2vect = new Word2Vec()
word2vec.setSeed(42)
val word2vecModel = word2vec.fit(tokens)

训练完成之后，很容易找到某个单词的前20个相近的词汇（也就是通过对词向量计算余弦相似度得到的最相似度的单词）。例如，使用下面的代码找到hockey最相似的20个单词：  
输出结果大部分和hockey或其它运动主题相关：

In [ ]:
word2vecModel.findSynonyms("hockey", 20).foreach( println )

另一个例子，我们为legislation 找到如下20个近义词：  
输出结果大部分与管理、政策、商业特征显著相关：

In [ ]:
word2vecModel.findSynonyms("legislation", 20).foreach(println)